In [ ]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizVert_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,17166.15,0.10,0.13,-0.04,0.04,0.02,-0.00,-0.00
3,0.10,4525.27,-0.11,-0.02,0.01,-0.00,0.00,-0.00
4,0.13,-0.11,1373.44,-0.01,0.00,0.01,0.00,0.00
5,-0.04,-0.02,-0.01,374.04,-0.00,-0.01,-0.00,-0.00
6,0.04,0.01,0.00,-0.00,90.55,0.00,0.00,0.00
7,0.02,-0.00,0.01,-0.01,0.00,20.86,-0.00,0.00
8,-0.00,0.00,0.00,-0.00,0.00,-0.00,4.54,0.00
9,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.85


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00001,0.00003,-0.00001,0.00003,0.00004,-0.00002,-0.00002
3,0.00001,1.00000,-0.00004,-0.00001,0.00002,-0.00001,0.00002,-0.00005
4,0.00003,-0.00004,1.00000,-0.00001,0.00001,0.00008,0.00000,0.00000
5,-0.00001,-0.00001,-0.00001,1.00000,-0.00003,-0.00006,-0.00001,-0.00001
6,0.00003,0.00002,0.00001,-0.00003,1.00000,0.00002,0.00002,0.00002
7,0.00004,-0.00001,0.00008,-0.00006,0.00002,1.00000,-0.00002,0.00010
8,-0.00002,0.00002,0.00000,-0.00001,0.00002,-0.00002,1.00000,0.00002
9,-0.00002,-0.00005,0.00000,-0.00001,0.00002,0.00010,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.2909756800585049

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.68955650e+07 2.51141339e+06 2.24671076e+05 1.60008241e+04
 8.80735728e+02 4.17285160e+01 1.50597318e+00 1.51612630e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999118,-0.040316,-0.011342,-0.003073,-0.000741,-0.000170,-0.000038,-0.000007
1,0.039573,0.997549,-0.055950,-0.013706,-0.003290,-0.000759,-0.000164,-0.000032
2,0.013320,0.054504,0.996563,-0.059416,-0.013168,-0.003009,-0.000657,-0.000123
3,0.004332,0.016514,0.057520,0.995989,-0.064832,-0.013911,-0.003013,-0.000565
4,0.001300,0.004938,0.016322,0.062512,0.994433,-0.081366,-0.016505,-0.003082
5,0.000395,0.001503,0.004939,0.018227,0.077827,0.990086,-0.113668,-0.019813
6,0.000126,0.000478,0.001575,0.005790,0.023929,0.106328,0.976201,-0.187396
7,0.000049,0.000185,0.000607,0.002233,0.009218,0.040000,0.183928,0.982080


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0008824309754589255,
 0.0024508696825593645,
 0.0034365462888128473,
 0.004011097506609329,
 0.0055667632573362935,
 0.009913705478065582,
 0.023799145354551188,
 0.017920489332651335]